In [30]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [31]:
module_path

'/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation'

In [3]:
import numpy as np
#standard Qiskit libraries
import qiskit
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer

In [33]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.0', 'qiskit-aer': '0.11.0', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.0', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [167]:
molecule = 'H 0. 0. 0.; F 0. 0. 0.9'
driver = PySCFDriver(atom=molecule)

In [169]:
qmolecule = driver.run()

In [170]:
fct = [FreezeCoreTransformer(freeze_core=False)] #[FreezeCoreTransformer(freeze_core=True, remove_orbitals=[3,4])]#[FreezeCoreTransformer(freeze_core=False)] 

In [171]:
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
problem = ElectronicStructureProblem(driver, fct)
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)

# Mapping Fermions to Qubits
#num_particles = (problem.molecule_data_transformed.num_alpha,
             #problem.molecule_data_transformed.num_beta)
num_particles = problem.num_particles
qubit_op = converter.convert(main_op, num_particles = problem.num_particles)
from qiskit_nature.circuit.library import HartreeFock


num_spin_orbitals = problem.num_spin_orbitals
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

In [172]:
help(init_state)

Help on HartreeFock in module qiskit_nature.circuit.library.initial_states.hartree_fock object:

class HartreeFock(qiskit.circuit.quantumcircuit.QuantumCircuit)
 |  HartreeFock(num_spin_orbitals: int, num_particles: Tuple[int, int], qubit_converter: qiskit_nature.converters.second_quantization.qubit_converter.QubitConverter) -> None
 |  
 |  A Hartree-Fock initial state.
 |  
 |  Method resolution order:
 |      HartreeFock
 |      qiskit.circuit.quantumcircuit.QuantumCircuit
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, num_spin_orbitals: int, num_particles: Tuple[int, int], qubit_converter: qiskit_nature.converters.second_quantization.qubit_converter.QubitConverter) -> None
 |      Args:
 |          num_spin_orbitals: The number of spin orbitals, has a min. value of 1.
 |          num_particles: The number of particles as a tuple storing the number of alpha- and
 |                         beta-spin electrons in the first and second number, respectivel

In [173]:
# second_q_ops['ParticleNumber']

In [174]:
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

# Parameters for twolocal su2 antatze
num_particles = problem.num_particles
num_spin_orbitals = problem.num_spin_orbitals
n = qubit_op.num_qubits
qc = QuantumCircuit(qubit_op.num_qubits)
from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister
#the variational parameter
p=1
qubit_label = 0
# qc.ry(theta, range(n))
#qc.rz(theta, range(n))
for k in range(1):
    for i in range(n):
        theta = Parameter(f"ry_in{p}" )
        qc.ry(theta, i)
        p += 1
    for i in range(n):
        theta = Parameter(f"rz_in{p}" )
        qc.rz(theta, i)
        p += 1
#     for i in range(n):
#         theta = Parameter(f"rz_theta{p}" )
#         qc.rx(theta, i)
#         p += 1
    for i in range(n-1):
        qc.cz(i, i+1)
    for i in range(n):
        theta = Parameter(f"ry_out{p}" )
        qc.ry(theta, i)
        p += 1
    for i in range(n):
        theta = Parameter(f"rz_out{p}" )
        qc.rz(theta, i)
        p += 1
#     for i in range(n):
#         theta = Parameter(f"rz_theta{p}" )
#         qc.rx(theta, i)
#         p += 1
#qc.rz(theta, range(n))

# Add the initial state
ansatz = qc
ansatz.compose(init_state, front=True, inplace=True)

In [175]:
qc.draw()

┌───┐      ┌────────────┐ ┌────────────┐   ┌──────────────┐»
q_0: ────┤ X ├──────┤ Ry(ry_in1) ├─┤ Rz(rz_in9) ├─■─┤ Ry(ry_out17) ├»
     ┌───┴───┴────┐┌┴────────────┤ └────────────┘ │ └──────────────┘»
q_1: ┤ Ry(ry_in2) ├┤ Rz(rz_in10) ├────────────────■────────■────────»
     └───┬───┬────┘└┬────────────┤┌─────────────┐          │        »
q_2: ────┤ X ├──────┤ Ry(ry_in3) ├┤ Rz(rz_in11) ├──────────■────────»
     ┌───┴───┴────┐┌┴────────────┤└─────────────┘                   »
q_3: ┤ Ry(ry_in4) ├┤ Rz(rz_in12) ├──────────────────────────────────»
     └───┬───┬────┘└┬────────────┤┌─────────────┐                   »
q_4: ────┤ X ├──────┤ Ry(ry_in5) ├┤ Rz(rz_in13) ├───────────────────»
     ┌───┴───┴────┐┌┴────────────┤└─────────────┘                   »
q_5: ┤ Ry(ry_in6) ├┤ Rz(rz_in14) ├──────────────────────────────────»
     └───┬───┬────┘└┬────────────┤┌─────────────┐                   »
q_6: ────┤ X ├──────┤ Ry(ry_in7) ├┤ Rz(rz_in15) ├───────────────────»
     ┌───┴───┴────┐┌┴────────────┤└─────────────┘                   »
q_7: ┤ Ry(ry_in8) ├┤ Rz(rz_in16) ├──────────────────────────────────»
     └────────────┘└─────────────┘                                  »
«     ┌──────────────┐                                                »
«q_0: ┤ Rz(rz_out25) ├────────────────────────────────────────────────»
«     ├──────────────┤┌──────────────┐                                »
«q_1: ┤ Ry(ry_out18) ├┤ Rz(rz_out26) ├────────────────────────────────»
«     └──────────────┘├──────────────┤┌──────────────┐                »
«q_2: ───────■────────┤ Ry(ry_out19) ├┤ Rz(rz_out27) ├────────────────»
«            │        └──────────────┘├──────────────┤┌──────────────┐»
«q_3: ───────■───────────────■────────┤ Ry(ry_out20) ├┤ Rz(rz_out28) ├»
«                            │        └──────────────┘├──────────────┤»
«q_4: ───────────────────────■───────────────■────────┤ Ry(ry_out21) ├»
«                                            │        └──────────────┘»
«q_5: ───────────────────────────────────────■───────────────■────────»
«                                                            │        »
«q_6: ───────────────────────────────────────────────────────■────────»
«                                                                     »
«q_7: ────────────────────────────────────────────────────────────────»
«                                                                     »
«                                                     
«q_0: ────────────────────────────────────────────────
«                                                     
«q_1: ────────────────────────────────────────────────
«                                                     
«q_2: ────────────────────────────────────────────────
«                                                     
«q_3: ────────────────────────────────────────────────
«     ┌──────────────┐                                
«q_4: ┤ Rz(rz_out29) ├────────────────────────────────
«     ├──────────────┤┌──────────────┐                
«q_5: ┤ Ry(ry_out22) ├┤ Rz(rz_out30) ├────────────────
«     └──────────────┘├──────────────┤┌──────────────┐
«q_6: ───────■────────┤ Ry(ry_out23) ├┤ Rz(rz_out31) ├
«            │        ├──────────────┤├──────────────┤
«q_7: ───────■────────┤ Ry(ry_out24) ├┤ Rz(rz_out32) ├
«                     └──────────────┘└──────────────┘

In [176]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
print(result_exact)

Exact electronic energy -28.057676178355965
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -103.884726386112
  - computed part:      -28.057676178356
  - FreezeCoreTransformer extracted energy part: -75.827050207756
~ Nuclear repulsion energy (Hartree): 5.2917721092
> Total ground state energy (Hartree): -98.592954276912
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 8.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  15.30678161]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  15.77038057]
    - computed part:      [0.0  0.0  12.36943963]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  3.40094094]
  > Dipole moment (a.u.): [0.0  0.0  -0.46359896]  Total: 0.46359896
                 (debye): [0.0  0.0  -1.17835091]  Total: 1.17835091
 


In [177]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
optimizer = COBYLA(maxiter=15000)
from qiskit.algorithms import VQE
from IPython.display import display, clear_output
def callback(eval_count, parameters, mean, std):  
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 600,
    'eigenstate': array([ 2.35207103e-20+2.64389397e-20j,  3.72214527e-15-1.34390683e-15j,
        2.43443304e-25-1.79927197e-25j, -2.53349947e-20-2.38992194e-20j,
       -6.61921264e-16-1.76052334e-15j, -2.10288842e-10-4.40246493e-12j,
       -8.23019538e-21-1.17233579e-20j, -9.76258597e-16+1.54272927e-15j,
       -3.36117364e-26+6.00332242e-25j,  6.09358408e-20+2.84266856e-20j,
       -5.22357204e-30+5.76929469e-31j,  3.28935529e-25+3.93998188e-25j,
        3.94239973e-20+3.48945337e-21j,  1.99991463e-15-3.94841601e-15j,
        2.97360045e-25-4.92638475e-26j, -2.03639782e-20-3.25759091e-20j,
        8.37928932e-16+7.34310903e-16j,  1.11049092e-10-5.64994336e-11j,
        6.90423593e-21-6.57054173e-21j, -8.84709349e-16-6.47890003e-16j,
       -2.75442502e-11-5.24222550e-11j, -6.58710329e-06+6.82363923e-07j,
       -3.02840352e-16-3.34177201e-16j, -2.44856695e-11+5.20050322e-11j,
       -4.81681029e-21-1.83076242e-2

In [178]:
ansatz.draw()

┌───┐      ┌────────────┐ ┌────────────┐   ┌──────────────┐»
q_0: ────┤ X ├──────┤ Ry(ry_in1) ├─┤ Rz(rz_in9) ├─■─┤ Ry(ry_out17) ├»
     ┌───┴───┴────┐┌┴────────────┤ └────────────┘ │ └──────────────┘»
q_1: ┤ Ry(ry_in2) ├┤ Rz(rz_in10) ├────────────────■────────■────────»
     └───┬───┬────┘└┬────────────┤┌─────────────┐          │        »
q_2: ────┤ X ├──────┤ Ry(ry_in3) ├┤ Rz(rz_in11) ├──────────■────────»
     ┌───┴───┴────┐┌┴────────────┤└─────────────┘                   »
q_3: ┤ Ry(ry_in4) ├┤ Rz(rz_in12) ├──────────────────────────────────»
     └───┬───┬────┘└┬────────────┤┌─────────────┐                   »
q_4: ────┤ X ├──────┤ Ry(ry_in5) ├┤ Rz(rz_in13) ├───────────────────»
     ┌───┴───┴────┐┌┴────────────┤└─────────────┘                   »
q_5: ┤ Ry(ry_in6) ├┤ Rz(rz_in14) ├──────────────────────────────────»
     └───┬───┬────┘└┬────────────┤┌─────────────┐                   »
q_6: ────┤ X ├──────┤ Ry(ry_in7) ├┤ Rz(rz_in15) ├───────────────────»
     ┌───┴───┴────┐┌┴────────────┤└─────────────┘                   »
q_7: ┤ Ry(ry_in8) ├┤ Rz(rz_in16) ├──────────────────────────────────»
     └────────────┘└─────────────┘                                  »
«     ┌──────────────┐                                                »
«q_0: ┤ Rz(rz_out25) ├────────────────────────────────────────────────»
«     ├──────────────┤┌──────────────┐                                »
«q_1: ┤ Ry(ry_out18) ├┤ Rz(rz_out26) ├────────────────────────────────»
«     └──────────────┘├──────────────┤┌──────────────┐                »
«q_2: ───────■────────┤ Ry(ry_out19) ├┤ Rz(rz_out27) ├────────────────»
«            │        └──────────────┘├──────────────┤┌──────────────┐»
«q_3: ───────■───────────────■────────┤ Ry(ry_out20) ├┤ Rz(rz_out28) ├»
«                            │        └──────────────┘├──────────────┤»
«q_4: ───────────────────────■───────────────■────────┤ Ry(ry_out21) ├»
«                                            │        └──────────────┘»
«q_5: ───────────────────────────────────────■───────────────■────────»
«                                                            │        »
«q_6: ───────────────────────────────────────────────────────■────────»
«                                                                     »
«q_7: ────────────────────────────────────────────────────────────────»
«                                                                     »
«                                                     
«q_0: ────────────────────────────────────────────────
«                                                     
«q_1: ────────────────────────────────────────────────
«                                                     
«q_2: ────────────────────────────────────────────────
«                                                     
«q_3: ────────────────────────────────────────────────
«     ┌──────────────┐                                
«q_4: ┤ Rz(rz_out29) ├────────────────────────────────
«     ├──────────────┤┌──────────────┐                
«q_5: ┤ Ry(ry_out22) ├┤ Rz(rz_out30) ├────────────────
«     └──────────────┘├──────────────┤┌──────────────┐
«q_6: ───────■────────┤ Ry(ry_out23) ├┤ Rz(rz_out31) ├
«            │        ├──────────────┤├──────────────┤
«q_7: ───────■────────┤ Ry(ry_out24) ├┤ Rz(rz_out32) ├
«                     └──────────────┘└──────────────┘

In [179]:
trans_ansatz = transpile(ansatz, optimization_level = 3)

In [180]:
parameter = result.optimal_parameters

In [181]:
parameter

{Parameter(ry_in1): 6.827165202923369e-06,
 Parameter(ry_in2): -1.6337052388981617e-05,
 Parameter(ry_in3): -1.9504648473842978e-05,
 Parameter(ry_in4): -4.583505307611714e-05,
 Parameter(ry_in5): 6.10354282870864e-05,
 Parameter(ry_in6): 5.965270087646158e-06,
 Parameter(ry_in7): 9.312744374460134e-06,
 Parameter(ry_in8): -0.0043764639879769,
 Parameter(ry_out17): 1.7553477050418194e-05,
 Parameter(ry_out18): -5.2173015630282704e-06,
 Parameter(ry_out19): 2.7300161977647244e-05,
 Parameter(ry_out20): 1.6108036965651842e-06,
 Parameter(ry_out21): -8.147972934398958e-06,
 Parameter(ry_out22): -8.65422271101483e-06,
 Parameter(ry_out23): -8.145059343430974e-06,
 Parameter(ry_out24): -0.0044055233224810164,
 Parameter(rz_in10): 1.5283131502120553,
 Parameter(rz_in11): 1.8467840590281954,
 Parameter(rz_in12): 1.9183949883797635,
 Parameter(rz_in13): 1.8180451683566035,
 Parameter(rz_in14): 1.8290020607579571,
 Parameter(rz_in15): 1.7186918294701956,
 Parameter(rz_in16): 0.00058092794910657

In [182]:
op_list = qubit_op.primitive.group_commuting()

In [183]:
help(qubit_op.primitive)

Help on SparsePauliOp in module qiskit.quantum_info.operators.symplectic.sparse_pauli_op object:

class SparsePauliOp(qiskit.quantum_info.operators.linear_op.LinearOp)
 |  SparsePauliOp(data, coeffs=None, *, ignore_pauli_phase=False, copy=True)
 |  
 |  Sparse N-qubit operator in a Pauli basis representation.
 |  
 |  This is a sparse representation of an N-qubit matrix
 |  :class:`~qiskit.quantum_info.Operator` in terms of N-qubit
 |  :class:`~qiskit.quantum_info.PauliList` and complex coefficients.
 |  
 |  It can be used for performing operator arithmetic for hundred of qubits
 |  if the number of non-zero Pauli basis terms is sufficiently small.
 |  
 |  The Pauli basis components are stored as a
 |  :class:`~qiskit.quantum_info.PauliList` object and can be accessed
 |  using the :attr:`~SparsePauliOp.paulis` attribute. The coefficients
 |  are stored as a complex Numpy array vector and can be accessed using
 |  the :attr:`~SparsePauliOp.coeffs` attribute.
 |  
 |  .. rubric:: Data

In [184]:
len(qubit_op.primitive.paulis)

276

In [185]:
paulis = qubit_op.primitive.paulis

In [186]:
qubit_op.primitive.paulis[0].to_label()

'IIZXIIZX'

In [187]:
help(qubit_op.primitive.paulis[0])

Help on Pauli in module qiskit.quantum_info.operators.symplectic.pauli object:

class Pauli(qiskit.quantum_info.operators.symplectic.base_pauli.BasePauli)
 |  Pauli(data=None, x=None, *, z=None, label=None)
 |  
 |  N-qubit Pauli operator.
 |  
 |  This class represents an operator :math:`P` from the full :math:`n`-qubit
 |  *Pauli* group
 |  
 |  .. math::
 |  
 |      P = (-i)^{q} P_{n-1} \otimes ... \otimes P_{0}
 |  
 |  where :math:`q\in \mathbb{Z}_4` and :math:`P_i \in \{I, X, Y, Z\}`
 |  are single-qubit Pauli matrices:
 |  
 |  .. math::
 |  
 |      I = \begin{pmatrix} 1 & 0  \\ 0 & 1  \end{pmatrix},
 |      X = \begin{pmatrix} 0 & 1  \\ 1 & 0  \end{pmatrix},
 |      Y = \begin{pmatrix} 0 & -i \\ i & 0  \end{pmatrix},
 |      Z = \begin{pmatrix} 1 & 0  \\ 0 & -1 \end{pmatrix}.
 |  
 |  **Initialization**
 |  
 |  A Pauli object can be initialized in several ways:
 |  
 |      ``Pauli(obj)``
 |          where ``obj`` is a Pauli string, ``Pauli`` or
 |          :class:`~qiskit.q

In [188]:
qubit_op.primitive.coeffs

array([ 3.95784589e-02+0.j, -3.95784589e-02+0.j, -3.95784589e-02+0.j,
        3.95784589e-02+0.j,  1.00781921e-02+0.j,  1.00781921e-02+0.j,
       -1.00781921e-02+0.j, -1.00781921e-02+0.j,  2.34287720e-02+0.j,
        2.34287720e-02+0.j, -2.34287720e-02+0.j, -2.34287720e-02+0.j,
        9.38033907e-02+0.j, -9.38033907e-02+0.j, -1.85969999e-02+0.j,
        1.85969999e-02+0.j,  1.92566058e-02+0.j, -1.92566058e-02+0.j,
        1.92566058e-02+0.j, -1.92566058e-02+0.j,  1.38853385e-02+0.j,
       -1.38853385e-02+0.j,  2.91139363e-03+0.j, -2.91139363e-03+0.j,
       -3.80217361e-03+0.j,  3.80217361e-03+0.j,  8.04713551e-03+0.j,
       -8.04713551e-03+0.j,  8.04713551e-03+0.j, -8.04713551e-03+0.j,
        4.47826708e-02+0.j,  4.47826708e-02+0.j,  4.47826708e-02+0.j,
        4.47826708e-02+0.j,  1.12094703e-02+0.j, -1.12094703e-02+0.j,
        1.12094703e-02+0.j, -1.12094703e-02+0.j, -1.10761002e-02+0.j,
       -1.10761002e-02+0.j, -1.10761002e-02+0.j, -1.10761002e-02+0.j,
        4.47826708e-

In [189]:
coeffs = qubit_op.primitive.coeffs

In [190]:
qubit_op.primitive.coeffs[0].real

0.03957845891262028

In [191]:
with open(r'HF_hamiltonian.txt', 'w') as fp:
    for i in range(len(paulis)):
        pauli_str = paulis[i].to_label()
        coeffs_str = str(coeffs[i].real)
        fp.write("{}, {}\n".format(pauli_str, coeffs_str))
    print('Done')

Done


In [192]:
pauli_list = read_from_file('HF_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [193]:
len(pauli_commute)

76